# deque - Double-Ended Queue

A double-ended queue, or deque, supports adding and removing elements from either end of the queue. The more commonly used stacks and queues are degenerate forms of deques, where the inputs and outputs are restricted to a single end.

Since deques are a type of sequence container, they support some of the same operations as list, such as examining the contents with \__getitem\__(), determining length, and removing elements from the middle of the queue by matching identity.

In [1]:
# collections_deque.py
import collections

d = collections.deque('abcdefg')
print('Deque:', d)
print('Length:', len(d))
print('Left end:', d[0])
print('Right end:', d[-1])

d.remove('c')
print('remove(c):', d)

Deque: deque(['a', 'b', 'c', 'd', 'e', 'f', 'g'])
Length: 7
Left end: a
Right end: g
remove(c): deque(['a', 'b', 'd', 'e', 'f', 'g'])


## Populating

A deque can be populated from either end, termed “left” and “right” in the Python implementation.

The extendleft() function iterates over its input and performs the equivalent of an appendleft() for each item. The end result is that the deque contains the input sequence in reverse order.

In [2]:
# collections_deque_populating.py
import collections

# Add to the right
print('Add to the right')
d1 = collections.deque()
d1.extend('abcdefg')
print('extend    :', d1)
d1.append('h')
print('append    :', d1)


# Add to the left
print('\nAdd to the left')
d2 = collections.deque()
d2.extendleft(range(6))
print('extendleft:', d2)
d2.appendleft(6)
print('appendleft:', d2)

Add to the right
extend    : deque(['a', 'b', 'c', 'd', 'e', 'f', 'g'])
append    : deque(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h'])

Add to the left
extendleft: deque([5, 4, 3, 2, 1, 0])
appendleft: deque([6, 5, 4, 3, 2, 1, 0])


#### What is the difference between the list methods append and extend?

http://stackoverflow.com/questions/252703/append-vs-extend

* append 

adds its argument as a single element to the end of a list. The length of the list itself will increase by one.

* extend 

iterates over its argument adding each element to the list, extending the list. The length of the list will increase by however many elements were in the iterable argument.

#### Operator Overload, __and__, (+) and __iand__ (+=)

Both + and += operators are defined for list. They are semantically similar to extend.

my_list + another_list creates a third list in memory, so you can return the result of it, but it requires that the second iterable be a list.

my_list += another_list modifies the list in-place (it is the in-place operator, and lists are mutable objects, as we've seen) so it does not create a new list. It also works like extend, in that the second iterable can be any kind of iterable.

#### Time Complexity

Append has constant time complexity, O(1).

Extend has time complexity, O(k).

Iterating through the multiple calls to append adds to the complexity, making it equivalent to that of extend, and since extend's iteration is implemented in C, it will always be faster if you intend to append successive items from an iterable onto a list

#### Conclusion

We see that extend can run much faster than append, and it is semantically clearer, so it is preferred when you intend to append each element in an iterable to a list.

If you only have a single element to add to the list, use append.

## Consuming

Similarly, the elements of the deque can be consumed from both ends or either end, depending on the algorithm being applied.

Use pop() to remove an item from the “right” end of the deque and popleft() to take an item from the “left” end.

In [3]:
# collections_deque_consuming.py
import collections

print('From the right:')
d = collections.deque('abcdefg')
print('d: ',d)
while True:
    try:
        print(d.pop(), end='')
    except IndexError:
        break
print()

print('\nFrom the left:')
d = collections.deque(range(6))
print('d: ',d)
while True:
    try:
        print(d.popleft(), end='')
    except IndexError:
        break

From the right:
d:  deque(['a', 'b', 'c', 'd', 'e', 'f', 'g'])
gfedcba

From the left:
d:  deque([0, 1, 2, 3, 4, 5])
012345

Since deques are thread-safe, the contents can even be consumed from both ends at the same time from separate threads.

The threads in this example alternate between each end, removing items until the deque is empty.

In [4]:
# collections_deque_both_ends.py
import collections
import threading
import time

candle = collections.deque(range(7))


def burn(direction, nextSource):
    while True:
        try:
            next = nextSource()
        except IndexError:
            break
        else:
            print('{:>8}: {}'.format(direction, next))
            time.sleep(0.3)
    print('{:>8} done'.format(direction))
    return

left = threading.Thread(target=burn,
                        args=('Left', candle.popleft))
right = threading.Thread(target=burn,
                         args=('Right', candle.pop))

left.start()
time.sleep(0.1)
right.start()

left.join()
right.join()

    Left: 0
   Right: 6
    Left: 1
   Right: 5
    Left: 2
   Right: 4
    Left: 3
   Right done
    Left done


## Rotating

Another useful aspect of the deque is the ability to rotate it in either direction, so as to skip over some items.

Rotating the deque to the right (using a positive rotation) takes items from the right end and moves them to the left end. Rotating to the left (with a negative value) takes items from the left end and moves them to the right end. It may help to visualize the items in the deque as being engraved along the edge of a dial.

In [5]:
# collections_deque_rotate.py
import collections

d = collections.deque(range(10))
print('Normal        :', d)

d = collections.deque(range(10))
d.rotate(2)
print('Right rotation:', d)

d = collections.deque(range(10))
d.rotate(-2)
print('Left rotation :', d)

Normal        : deque([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
Right rotation: deque([8, 9, 0, 1, 2, 3, 4, 5, 6, 7])
Left rotation : deque([2, 3, 4, 5, 6, 7, 8, 9, 0, 1])


## Constraining the Queue Size

A deque instance can be configured with a maximum length so that it never grows beyond that size. When the queue reaches the specified length, existing items are discarded as new items are added. This behavior is useful for finding the last n items in a stream of undetermined length.

The deque length is maintained regardless of which end the items are added to.

In [6]:
# collections_deque_maxlen.py
import collections
import random

# Set the random seed so we see the same output each time
# the script is run.
random.seed(1)

d1 = collections.deque(maxlen=3)
d2 = collections.deque(maxlen=3)

for i in range(5):
    n = random.randint(0, 100)
    print('\nn =', n)
    d1.append(n)
    d2.appendleft(n)
    print('D1:', d1)
    print('D2:', d2)


n = 17
D1: deque([17], maxlen=3)
D2: deque([17], maxlen=3)

n = 72
D1: deque([17, 72], maxlen=3)
D2: deque([72, 17], maxlen=3)

n = 97
D1: deque([17, 72, 97], maxlen=3)
D2: deque([97, 72, 17], maxlen=3)

n = 8
D1: deque([72, 97, 8], maxlen=3)
D2: deque([8, 97, 72], maxlen=3)

n = 32
D1: deque([97, 8, 32], maxlen=3)
D2: deque([32, 8, 97], maxlen=3)
